# 01 - Exploración de Diccionarios y Fuentes

Este notebook detecta y lista archivos relevantes en `/mnt/data` y sugiere cómo mapear variables clave (SEACE, SIAF, Invierte, INFOBRAS).

In [ ]:
from pathlib import Path
import pandas as pd
base = Path('/mnt/data')
files = list(base.glob('*.xlsx')) + list(base.glob('*.xls')) + list(base.glob('*.csv'))
files

In [ ]:
dicc_candidates = [f for f in files if 'Diccionario' in f.name or 'Perfilamiento' in f.name]
dicc_candidates

In [ ]:
dfs = {}
for f in dicc_candidates[:3]:
    try:
        df = pd.read_excel(f)
        dfs[f.name] = df.head(10)
    except Exception as e:
        print('No se pudo leer', f, e)
dfs

## Próximos pasos
- Copiar diccionarios a `data/external/`.
- Diseñar *mapping* de campos → dataset unificado.
- Definir `target` (variable de riesgo) y reglas de etiquetado.
- Generar `data/processed/dataset_obras.parquet`.